In [19]:
import pandas as pd
import numpy as np
import os

## Split total table

In [20]:
dirlist = ["2018","2019","2020","2021"]

In [21]:
shpfiles = []
for inputdir in dirlist:
        print(inputdir)
        subfiles = []
        for dirpath, subdirs, files in os.walk(inputdir+"/"):
            for x in files:
                if x.endswith(".gz"):
                    subfiles.append(os.path.join(dirpath, x))
        shpfiles.append(subfiles)

2018
2019
2020
2021


In [22]:
def vwap(data):
    #data.dropna(inplace=True)
    if data.volume.sum() == 0:
        return np.nan
    else: 
        return (data.price * data.volume).sum() / data.volume.sum()

In [23]:
def getPrice(sub_data):
    price_df = sub_data.groupby([pd.Grouper(freq='min')]).min()
    price_df.rename(columns = {"price":"low_price"}, inplace = True)
    price_df["volume"] = sub_data["volume"].groupby([pd.Grouper(freq='min')]).sum().tolist()
    price_df["high_price"] = sub_data["price"].groupby([pd.Grouper(freq='min')]).max().tolist()
    price_df["open_price"] = sub_data["price"].groupby([pd.Grouper(freq='min')]).first().tolist()
    price_df["close_price"] = sub_data["price"].groupby([pd.Grouper(freq='min')]).last().tolist()
    price_df["vmap"] = sub_data.groupby([pd.Grouper(freq='min')], group_keys=False).apply(vwap).tolist()
    price_df.dropna(how="any",inplace=True)
    #price_df.reset_index(inplace=True)
    #price_df["datetime"] = price_df["date"] +" "+ price_df["minete"]+":00"
    return price_df[["volume","low_price","high_price","open_price","close_price","vmap"]]

In [24]:
for fileitem in shpfiles:
#     li = []
#     for filename in fileitem:
#         df =pd.read_csv(filename,usecols=[0,1,6,7,9],names=["date","time","day","volume","price"], header=None, sep=',', quotechar='"')
#         li.append(df)
#     frame = pd.concat(li, axis=0, ignore_index=True)
    for file in fileitem:
        data = pd.read_csv(file,usecols=[0,1,6,7,9],parse_dates=[[0, 1]],header=None, sep=',',quotechar='"')
        #data.rename =["date","time","volume","price"]
        data.rename(columns={'0_1':'datetime', 6: 'day', 7: 'volume', 9:'price'},inplace=True)
        data.set_index("datetime",inplace=True)
        day_m = pd.unique(data['day']).tolist()
        for item in day_m:
            subdf = data[data["day"]== item]
            price_df = getPrice(subdf)
        #subdf.to_csv("1502_"+item+".csv",header=True,index=False)
        #subdf["minete"] = subdf["time"].astype(str).str[0:5]
#             subdf.drop(columns=["time"],inplace=True)
            price_df = getPrice(subdf)
            savepath = "result"+"/"+ file[:10]
            os.makedirs(savepath, exist_ok=True)
            price_df.to_csv(savepath+"/"+str(item)+"price.csv",header=True, index=True, float_format='%.3f')

In [113]:
price_df

,volume,low_price,high_price,open_price,close_price,vmap
datetime,,,,,,
2015-03-03 00:00:00,8,2.676,2.676,2.676,2.676,2.67600
2015-03-03 00:01:00,2,2.675,2.677,2.677,2.675,2.67600
2015-03-03 00:03:00,1,2.677,2.677,2.677,2.677,2.67700
2015-03-03 00:04:00,3,2.675,2.675,2.675,2.675,2.67500
2015-03-03 00:05:00,1,2.676,2.676,2.676,2.676,2.67600
...,...,...,...,...,...,...
2015-03-03 23:55:00,1,2.677,2.677,2.677,2.677,2.67700
2015-03-03 23:56:00,4,2.676,2.677,2.677,2.676,2.67675
2015-03-03 23:57:00,1,2.677,2.677,2.677,2.677,2.67700


In [25]:
import glob

In [31]:
CONCAT_DIR = "result/globel/"
os.makedirs(CONCAT_DIR, exist_ok=True)

for item in dirlist:
    # Use glob module to return all csv files under root directory. Create DF from this.
    files = pd.DataFrame([file for file in glob.glob("result/"+item+"/*/*/*")], columns=["fullpath"])

    #    fullpath
    # 0  root\dir1\data_20170101_k.csv
    # 1  root\dir1\data_20170102_k.csv
    # 2  root\dir2\data_20170101_k.csv
    # 3  root\dir2\data_20170102_k.csv

    # # Split the full path into directory and filename
    files_split = files['fullpath'].str.rsplit("/", 1, expand=True).rename(columns={0: 'path', 1:'filename'})

    #    path       filename
    # 0  root\dir1  data_20170101_k.csv
    # 1  root\dir1  data_20170102_k.csv
    # 2  root\dir2  data_20170101_k.csv
    # 3  root\dir2  data_20170102_k.csv

    # Join these into one DataFrame
    files = files.join(files_split)
    print(files)

    #    fullpath                       path        filename
    # 0  root\dir1\data_20170101_k.csv  root\dir1   data_20170101_k.csv
    # 1  root\dir1\data_20170102_k.csv  root\dir1   data_20170102_k.csv
    # 2  root\dir2\data_20170101_k.csv  root\dir2   data_20170101_k.csv
    # 3  root\dir2\data_20170102_k.csv  root\dir2   data_20170102_k.csv

#     # Iterate over unique filenames; read CSVs, concat DFs, save file
    for f in files['filename'].unique():
        paths = files[files['filename'] == f]['fullpath'] # Get list of fullpaths from unique filenames
        dfs = [pd.read_csv(path) for path in paths] # Get list of dataframes from CSV file paths
        concat_df = pd.concat(dfs) # Concat dataframes into one 
        concat_df["CC_date"] = f[:4]
        savepath = CONCAT_DIR + item + "/"
        os.makedirs(savepath, exist_ok=True)
        concat_df.to_csv(savepath+f,index=False, float_format='%.3f') # Save dataframe

                             fullpath               path       filename
0     result/2018/03/05/1902price.csv  result/2018/03/05  1902price.csv
1     result/2018/03/05/1903price.csv  result/2018/03/05  1903price.csv
2     result/2018/03/05/2010price.csv  result/2018/03/05  2010price.csv
3     result/2018/03/05/2011price.csv  result/2018/03/05  2011price.csv
4     result/2018/03/05/1908price.csv  result/2018/03/05  1908price.csv
...                               ...                ...            ...
7396  result/2018/12/14/2008price.csv  result/2018/12/14  2008price.csv
7397  result/2018/12/14/1911price.csv  result/2018/12/14  1911price.csv
7398  result/2018/12/14/1910price.csv  result/2018/12/14  1910price.csv
7399  result/2018/12/14/2003price.csv  result/2018/12/14  2003price.csv
7400  result/2018/12/14/2002price.csv  result/2018/12/14  2002price.csv

[7401 rows x 3 columns]
                             fullpath               path       filename
0     result/2019/03/04/2201price.csv  

In [16]:
files

,fullpath


In [34]:
CONCAT_DIR = "result/globel/"
for item in dirlist:
    filelist = [file for file in glob.glob("result/globel/"+item+"/*")]
    dfs = [pd.read_csv(path) for path in filelist] # Get list of dataframes from CSV file paths
    concat_df = pd.concat(dfs) # Concat dataframes into one
    #concat_df.sort_index(inplace=True)
    concat_df.sort_values(by=['datetime'],inplace=True)
    savepath = CONCAT_DIR
    os.makedirs(savepath, exist_ok=True)
    concat_df.to_csv(savepath+ item +"_price.csv",index=False, float_format='%.3f') # Save dataframe